Model Training (Google Colab)

In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics
!pip install roboflow

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import yaml
from PIL import Image
from ultralytics import YOLO
from IPython.display import Video

In [ ]:
# Configure the visual appearance of Seaborn plots
sns.set(rc={'axes.facecolor': '#eae8fa'}, style='darkgrid')

In [ ]:
model = YOLO('yolov8s.pt')

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="") #https://docs.roboflow.com/developer/authentication/find-your-roboflow-api-key
project = rf.workspace("computer-vision-l3ex2").project("combined-dataset-cctsdb-500")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
#Confirm Dataset Labels
dataset_path = '/content/combined-dataset-cctsdb-500-1'

yaml_file_path = os.path.join(dataset_path, dataset_path + "/data.yaml")
with open(yaml_file_path, 'r') as file:
    yaml_content = yaml.load(file, Loader=yaml.FullLoader)
    print(yaml.dump(yaml_content, default_flow_style=False))

In [ ]:
# Confirm Dataset Size
train_images_path = os.path.join(dataset_path, 'train', 'images')
valid_images_path = os.path.join(dataset_path, 'valid', 'images')

num_train_images = 0
num_valid_images = 0

train_image_sizes = set()
valid_image_sizes = set()

for filename in os.listdir(train_images_path):
    if filename.endswith('.jpg'):
        num_train_images += 1
        image_path = os.path.join(train_images_path, filename)
        with Image.open(image_path) as img:
            train_image_sizes.add(img.size)

for filename in os.listdir(valid_images_path):
    if filename.endswith('.jpg'):
        num_valid_images += 1
        image_path = os.path.join(valid_images_path, filename)
        with Image.open(image_path) as img:
            valid_image_sizes.add(img.size)

print(f"Number of training images: {num_train_images}")
print(f"Number of validation images: {num_valid_images}")

In [ ]:
# Helper function to draw ground truth boxes
def draw_ground_truth(img_path, label_path, class_names):
    image = cv2.imread(img_path)
    h, w, _ = image.shape

    # If label file exists, draw the boxes
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                class_id, cx, cy, bw, bh = map(float, line.strip().split())
                x1 = int((cx - bw / 2) * w)
                y1 = int((cy - bh / 2) * h)
                x2 = int((cx + bw / 2) * w)
                y2 = int((cy + bh / 2) * h)
                label = class_names[int(class_id)]
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (36, 255, 12), 2)

    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Load class names from YAML
with open(dataset_path + "/data.yaml", 'r') as f:
    class_names = yaml.safe_load(f)['names']

# Get 8 image files evenly
image_files = [f for f in os.listdir(train_images_path) if f.endswith('.jpg')]
num_images = len(image_files)
selected_images = [image_files[i] for i in range(0, num_images, max(1, num_images // 8))][:8]

# Plot
fig, axes = plt.subplots(2, 4, figsize=(20, 11))
for ax, img_file in zip(axes.ravel(), selected_images):
    img_path = os.path.join(train_images_path, img_file)
    label_file = img_file.replace(".jpg", ".txt")
    label_path = os.path.join(train_images_path.replace("images", "labels"), label_file)

    annotated_image = draw_ground_truth(img_path, label_path, class_names)
    ax.imshow(annotated_image)
    clean_title = img_file.split('_')[0] + "..." + img_file[-8:-4]
    ax.set_title(clean_title, fontsize=10)
    ax.axis('off')

plt.suptitle("Sample Training Images with Ground Truth Annotations", fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
!pip install --upgrade ultralytics ray wandb

In [ ]:
results = model.train(
    data=yaml_file_path,
    epochs=100,
    imgsz=640,
    device=0,
    patience=20,
    batch=8,
    optimizer='AdamW',
    lr0=1e-4,
    lrf=0.01,
    dropout=0.2,
    seed=42
)

Saving Model

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define save path on Drive
drive_path = ""

!mkdir -p {drive_path}
!cp -r /content/runs/detect/train {drive_path}/
!ls -lh {drive_path}/train